In [3]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import calendar
import datetime
import requests
from bs4 import BeautifulSoup

In [4]:
path = os.path.join(os.getcwd(),"Data")
revenuepath = os.path.join(path,"Revenue")
# print(revenuepath)
# print(path)

In [5]:
# equity = pd.read_csv(os.path.join(path,"Equity.csv"))
# equity = equity[["Security Code","Security Id","Security Name"]]
# equity_samp = equity.head(7)
# equity_samp

In [6]:
url = "https://www.bseindia.com/corporates/results.aspx?Code=500003&Company=AEGIS+LOGISTICS+LTD.&qtr=104.00&RType=D"

In [7]:
def download_revenue_profit(code,name): 

    def create_driver():
        chromeOptions = webdriver.ChromeOptions()
        chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        return driver

    def download():
        columns = ["security code","security name",'total income','net sales/revenue from operations','expenditure','net profit','basic eps for continuing operation','basic & diluted eps before extraordinary items',"year","quartile"]
        code_df = pd.DataFrame(columns=columns)
        for q in range(55,104):
            url = "https://www.bseindia.com/corporates/results.aspx?Code=" + str(code) +"&Company="+ str(name) +"&qtr="+ str(q) +"&RType=D"
            driver.get(url)
            html = driver.page_source
            soup = BeautifulSoup(html, "html")

            table = soup.find_all("table",attrs={"id":"ContentPlaceHolder1_tbl_typeID"})
            table = pd.read_html(str(table))[0]
            table = table[[0,1]]
            table.dropna(inplace=True)
            table = table.transpose()
            table.columns = table.iloc[0]
            table = table[1:]
            table.columns = map(str.lower, table.columns)
            table.drop(["description"],inplace=True,axis=1)
            try:
                table["date begin"] = pd.to_datetime(table["date begin"])
                date = table.iloc[0]["date begin"]
                table["quartile"] =  (date.month-1)//3 + 1
                table["year"] = date.year
                table["security name"] = name
                table["security code"] = code
                code_df = code_df.append(table,ignore_index=True)
            except Exception as e:
                pass

        code_df["eps"] = code_df['basic eps for continuing operation'].fillna('') + code_df['basic & diluted eps before extraordinary items'].fillna('')
        keep_columns = ['security code', 'security name', 'total income','net sales/revenue from operations', 'expenditure', 'net profit','year', 'quartile','eps']
        res = ["other operating income","other income","net sales / income from operations","total income"]
        code_df[res] = code_df[res].apply(pd.to_numeric)
        code_df = code_df.fillna(0)
        code_df["total income"] = code_df["total income"] + code_df["other operating income"] + code_df["other income"] + code_df["net sales / income from operations"]
        code_df = code_df[keep_columns]
        code_df.to_csv(os.path.join(revenuepath,str(code)+".csv"),index=None)
    driver = create_driver()
    download()

In [8]:
# download_revenue_profit("500002","ABB India Limited")

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\VenkataSaiKrishna\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache
 
